In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
master_df = pd.read_csv('master_data_table.csv')
feed_df = pd.read_csv('./Example Inputs/feed1.csv')

df = master_df.merge(feed_df, how='outer', on='entry').dropna()

df.head()

,Unnamed: 0,wals code,name,value,description_x,latitude,longitude,area,title,entry,description_y
89,1.0,xoo,!Xóõ,5.0,Large,-24.000000,21.500000,Phonology,Consonant Inventories,Consonant Inventories 5,Large
90,2.0,ani,//Ani,5.0,Large,-18.916667,21.916667,Phonology,Consonant Inventories,Consonant Inventories 5,Large
91,4.0,abk,Abkhaz,5.0,Large,43.083333,41.000000,Phonology,Consonant Inventories,Consonant Inventories 5,Large
92,7.0,aco,Acoma,5.0,Large,34.916667,-107.583333,Phonology,Consonant Inventories,Consonant Inventories 5,Large
93,38.0,arc,Archi,5.0,Large,42.000000,46.833333,Phonology,Consonant Inventories,Consonant Inventories 5,Large


In [31]:
# Export list of locations of languages with these features.

language_list_df = df.reset_index()
language_list_df = language_list_df[["name","latitude","longitude"]]

names = []
drop_list = []

frequency = language_list_df["name"].value_counts()
language_list_df["count"] = 0

for i in range(len(language_list_df)):
    language_list_df["count"][i] = frequency[language_list_df["name"][i]]
    if language_list_df.iloc[i,0] in names:
        drop_list.append(i)
    else:
        names.append(language_list_df.iloc[i,0])

language_list_df.drop(drop_list, inplace = True)
language_list_df = language_list_df.reset_index()
language_list_df = language_list_df[["name","latitude","longitude","count"]]
language_list_df.to_csv('language_list_df.csv')

language_list_df

C:\Users\metatron\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,name,latitude,longitude,count
0,!Xóõ,-24.000000,21.500000,1
1,//Ani,-18.916667,21.916667,4
2,Abkhaz,43.083333,41.000000,5
3,Acoma,34.916667,-107.583333,6
4,Archi,42.000000,46.833333,4
...,...,...,...,...
1613,Zande,4.000000,26.000000,1
1614,Zapotec (Isthmus),16.416667,-95.000000,1
1615,Zapotec (Mitla),16.800000,-96.250000,1
1616,Zapotec (Zoogocho),17.250000,-96.250000,1


In [4]:
# # location = list(zip(df.latitude, df.longitude))
# df['latitude'] = latitude
df['entry'] = df['entry'].str.replace('/', ' or ')
df['entry'] = df['entry'].str.replace(' ', '_')

In [5]:
df

,Unnamed: 0,wals code,name,value,description_x,latitude,longitude,area,title,entry,description_y
89,1.0,xoo,!Xóõ,5.0,Large,-24.000000,21.500000,Phonology,Consonant Inventories,Consonant_Inventories_5,Large
90,2.0,ani,//Ani,5.0,Large,-18.916667,21.916667,Phonology,Consonant Inventories,Consonant_Inventories_5,Large
91,4.0,abk,Abkhaz,5.0,Large,43.083333,41.000000,Phonology,Consonant Inventories,Consonant_Inventories_5,Large
92,7.0,aco,Acoma,5.0,Large,34.916667,-107.583333,Phonology,Consonant Inventories,Consonant_Inventories_5,Large
93,38.0,arc,Archi,5.0,Large,42.000000,46.833333,Phonology,Consonant Inventories,Consonant_Inventories_5,Large
...,...,...,...,...,...,...,...,...,...,...,...
75316,75317.0,wag,Wagiman,1.0,Word&NoDoubleNeg,-14.000000,131.250000,Word Order,SONegV Order,SONegV_Order_1,Word&NoDoubleNeg
75317,75318.0,wak,Wakhi,1.0,Word&NoDoubleNeg,36.500000,72.000000,Word Order,SONegV Order,SONegV_Order_1,Word&NoDoubleNeg
75318,75326.0,wsk,Waskia,1.0,Word&NoDoubleNeg,-4.500000,146.000000,Word Order,SONegV Order,SONegV_Order_1,Word&NoDoubleNeg
75319,75337.0,yid,Yidiny,1.0,Word&NoDoubleNeg,-17.000000,145.750000,Word Order,SONegV Order,SONegV_Order_1,Word&NoDoubleNeg


In [6]:
# entry = df['entry']
# dummies_entry = pd.get_dummies(entry)
# df['dummies_entry'] = dummies_entry
# df

In [7]:
X = df['entry']
y = df['latitude']

X.shape, y.shape

((3324,), (3324,))

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(X)
encoded_X = label_encoder.transform(X)

In [9]:
for label, original_class in zip(encoded_X, X):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Cons

Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_Tense-Aspect_Affixes_2
Encoded Label: 5
------------
Original Class: Position_of_

------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Enco

------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morph

Original Class: The_Position_of_Negative_Morphemes_in_SVO_Languages_13
Encoded Label: 11
------------
Original Class: The_Position_of_Negative_Morphemes_in_SVO_Languages_13
Encoded Label: 11
------------
Original Class: The_Position_of_Negative_Morphemes_in_SVO_Languages_13
Encoded Label: 11
------------
Original Class: The_Position_of_Negative_Morphemes_in_SVO_Languages_13
Encoded Label: 11
------------
Original Class: The_Position_of_Negative_Morphemes_in_SVO_Languages_13
Encoded Label: 11
------------
Original Class: The_Position_of_Negative_Morphemes_in_SVO_Languages_13
Encoded Label: 11
------------
Original Class: The_Position_of_Negative_Morphemes_in_SVO_Languages_13
Encoded Label: 11
------------
Original Class: The_Position_of_Negative_Morphemes_in_SVO_Languages_13
Encoded Label: 11
------------
Original Class: The_Position_of_Negative_Morphemes_in_SVO_Languages_13
Encoded Label: 11
------------
Original Class: The_Position_of_Negative_Morphemes_in_SVO_Languages_13
Encoded Lab

In [10]:
one_hot_X = to_categorical(encoded_X)
# cols = [1,2,3,4,5,6,7,8,9,10]
# df = df[df.columns[cols]]
# df
one_hot_X[0:3,0:5]

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [11]:
# # Step 1: Label-encode data set
# label_encoder = LabelEncoder()
# label_encoder.fit(y)
# encoded_y = label_encoder.transform(y)

In [12]:
# create training and testing data
X_train, X_test, y_train, y_test = train_test_split(one_hot_X, y, random_state=1)

In [13]:
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
classifier.fit(one_hot_X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
from sklearn.metrics import mean_squared_error, r2_score

# Use our model to predict a value
predicted = classifier.predict(one_hot_X)

# Score the prediction with mse and r2
mse = mean_squared_error(y, predicted)
r2 = r2_score(y, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

Mean Squared Error (MSE): 542.2197316041031
R-squared (R2 ): 0.035373162919849266


In [16]:
predicted_lat = classifier.predict(one_hot_X)

In [17]:
predicted_lat_avg = np.average(predicted_lat)
print(predicted_lat_avg)

12.910666


In [18]:
X = df['entry']
y = df['longitude']

X.shape, y.shape

((3324,), (3324,))

In [19]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(X)
encoded_X = label_encoder.transform(X)

In [20]:
for label, original_class in zip(encoded_X, X):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Consonant_Inventories_5
Encoded Label: 0
------------
Original Class: Cons

Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjunction_1
Encoded Label: 3
------------
Original Class: Noun_Phrase_Conjun

------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Encoded Label: 10
------------
Original Class: The_Morphological_Imperative_1
Enco

------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morph

------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morphological_means_of_signaling_negation_4
Encoded Label: 2
------------
Original Class: Minor_morph

In [21]:
one_hot_X = to_categorical(encoded_X)
one_hot_X[0:3,0:5]

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [22]:
# create training and testing data
X_train, X_test, y_train, y_test = train_test_split(one_hot_X, y, random_state=1)

In [23]:
classifier = LinearRegression()
classifier

classifier.fit(one_hot_X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [24]:
# Use our model to predict a value
predicted = classifier.predict(one_hot_X)

# Score the prediction with mse and r2
mse = mean_squared_error(y, predicted)
r2 = r2_score(y, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

predicted_lon = classifier.predict(one_hot_X)

predicted_lon_avg = np.average(predicted_lon)
print(predicted_lon_avg)

Mean Squared Error (MSE): 8092.103374228206
R-squared (R2 ): 0.015905591289003485
34.453735


In [25]:
print(str(predicted_lat_avg) + ', ' + str(predicted_lon_avg))

12.910666, 34.453735


In [26]:
location_df = pd.DataFrame()
location_df['Latitude'] = predicted_lat
location_df['Longitude'] = predicted_lon
location_df

,Latitude,Longitude
0,24.938023,10.255838
1,24.938023,10.255838
2,24.938023,10.255838
3,24.938023,10.255838
4,24.938023,10.255838
...,...,...
3319,17.182674,75.342087
3320,17.182674,75.342087
3321,17.182674,75.342087
3322,17.182674,75.342087


In [27]:
location_df.to_csv('location_list.csv')